<h2>HelloHistory- Learn History On the Go </h2>

In [1]:
#Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

#This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

#If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
#google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:2]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:2]}")
else:
    print("Anthropic API Key not set")



OpenAI API Key exists and begins sk
Anthropic API Key exists and begins sk


In [4]:
#connect to models

openai = OpenAI()

claude = anthropic.Anthropic()

In [5]:
system_message = "You are a helpful tutor teaching people history. You have to answer their questions on historical events."

In [6]:
#test function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [7]:

message_gpt("Who won World War II?")

'World War II was won by the Allies, a coalition of countries that included the United States, the Soviet Union, the United Kingdom, China, and several other nations. The war officially ended in 1945, with the unconditional surrender of Nazi Germany in May and the surrender of Japan in September.'

In [8]:

message_gpt("How did World War II begin?")

"World War II began on September 1, 1939, when Germany, led by Adolf Hitler, invaded Poland. This invasion was a result of aggressive expansionist policies pursued by Nazi Germany throughout the 1930s, which included the annexation of Austria and the incorporation of Czechoslovakia's Sudetenland.\n\nThe invasion of Poland prompted Britain and France to declare war on Germany on September 3, 1939, fulfilling their commitments to support Poland. Tensions had been building in Europe due to unresolved issues from World War I, the rise of totalitarian regimes, and various treaties and alliances. The war would expand rapidly as other nations became involved, eventually leading to a global conflict that lasted until 1945."

In [9]:
#add streaming 
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [11]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Ask HistoryBot a question:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [12]:
#streaming with Claude

def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [13]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Ask HistoryBot a question:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [14]:
# function to select model

def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [16]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Ask HistoryBot a question"), gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
